#### Introduction

This notebook creates maps and tables to determine the transit service at or near traffic signals. Currently, only signals on the State Highway Network (SHN) are shown.

#### How to use
On the calitp Jupyterhub, import `shared_utils` and authenticate to GCS.

#### Technical details
- Signal data is sourced from HQ Traffic Ops GIS, downloaded on 2025-09-08.
Segment data is calculated based on GTFS data collected by DDS for the selected targed date
- Routes are associated with a signal if they run within 155 meters of the signal. 
- Speeds and frequencies are defined based on data in the morning peak (7a-10a). Displayed speeds are the 20th percentile speeds.
- Network lines are based on data provided by Rebel. They are manually drawn, and do not strictly represent transit routes.
- Detailed speeds methodology is available at https://analysis.dds.dot.ca.gov/rt/README.html

#### Outputs
- Signal Map
  - Segments
    - 20th percentile speeds
    - Average frequencies
  - Signals
    - Unit Type
    - Aggregated data from 155 m to the signal
    - Route names (from `route_short_name`)
    - Organization names (from `organization_name`)
    - Total trips per hour, in both directions
    - IMMS ID
    - TMS ID
    - Delegation Type (describes whether a signal is owned/operated/maintaned by Caltrans or a local gov)
- Signal-Grain CSV
  - `signals_aggregated.csv`
    - Same data as in the interactive map
    - Additionally, the ArcGIS objectid is present to serve as a unique identifier
  - `signals_routes.csv`
    - One row for each route in each direction within 155 meters of each signal
    - Objectid
    - Route name
    - Organization name
    - Trips per hour in the chosen direction
    - TMS id
    - IMMS id
    - Delegation Type
    - Leased / Owned (whether the equipment is leased or owned by Caltrans)
    - Traffic Ops Comments
    
    
    

In [ ]:
import branca
import geopandas as gpd
from calitp_data_analysis import gcs_geopandas, geography_utils
from shared_utils import catalog_utils, rt_dates, rt_utils, webmap_utils
from signal_tools import (
    add_transit_metrics_to_signals,
    clean_sjoin_signals_segments,
    filter_points_along_corridor,
    ready_signals_for_display,
    ready_speedmap_segments_for_display,
)
from uris import (
    CALTRANS_INTERNAL_SIGNAL_URI,
    LOS_ANGELES_OPEN_SIGNAL_URI,
    SANTA_MONICA_OPEN_SIGNAL_URI,
)

In [ ]:
# constants
TARGET_DATE = rt_dates.DATES["jul2025"]
TARGET_TIME_OF_DAY = "AM Peak"
ANALYSIS_DISTRICT_NUMBER = 7
TARGET_TIME_OF_DAY_LENGTH_HOURS = (
    3  # the length of the target time of day (3 hours for am peak)
)
SJOIN_DISTANCE_METERS = 155

In [ ]:
# read geo files
shared_data_catalog = catalog_utils.get_catalog("shared_data_catalog")
gtfs_data_constants = catalog_utils.get_catalog("gtfs_analytics_data")

# Get district polygons to mask
districts = shared_data_catalog.caltrans_districts.read()
analysis_district = districts.loc[districts["DISTRICT"] == ANALYSIS_DISTRICT_NUMBER]

In [ ]:
g = gcs_geopandas.GCSGeoPandas()

### Get Signal Data

In [ ]:
# Get signal data from Caltrans Traffic Ops Data
caltrans_signals = (
    g.read_file(
        CALTRANS_INTERNAL_SIGNAL_URI,
    )
    .rename(columns=lambda s: s.lower())
    .clip(analysis_district)
)  # we want columns to be all lower case
# Filter out devices that aren't actually caltrans_signals
caltrans_signals_filtered = caltrans_signals.loc[
    caltrans_signals["tms_unit_type"] == "Traffic Signals"
].set_index("objectid")

In [ ]:
# Get signal data from LADOT Open Data

In [ ]:
# Get signal data from Santa Monica Open Data

### Get Speedmaps

In [ ]:
# Get speedmap data
speedmap_segments = g.read_parquet(
    f"{gtfs_data_constants.speedmap_segments.dir}{gtfs_data_constants.speedmap_segments.segment_timeofday}_{TARGET_DATE}.parquet",
    filters=[
        ("time_of_day", "=", TARGET_TIME_OF_DAY)
    ],  # Filter for only a selected time of day
).clip(analysis_district)

### Merge signal and speedmap info

In [ ]:
sjoined_signals_segments = clean_sjoin_signals_segments(
    signals_gdf=caltrans_signals_filtered,
    speedmap_segments_gdf=speedmap_segments,
    buffer_distance=SJOIN_DISTANCE_METERS,
)
signals_with_transit_metrics = add_transit_metrics_to_signals(
    signals_gdf=caltrans_signals_filtered,
    sjoined_signals_segments=sjoined_signals_segments,
)

### Get GDFs formatted for display on the webmap

In [ ]:
# Segment GDF
arrowized_gdf = ready_speedmap_segments_for_display(sjoined_signals_segments)
# Signal GDF
buffered_signals = ready_signals_for_display(
    signals_with_transit_metrics, buffer_distance=50
)

In [ ]:
# Define columns to include
signals_with_transit_display_columns = [
    "tms_unit_type",
    "route_names_aggregated",
    "organization_names_aggregated",
    "Trips/Hour",
    # "asset_sub_type",
    "tms_id",
    "imms_id",
    "delegation_type",
    # "leased_owned",
    # "comment",
    caltrans_signals_filtered.geometry.name,
]
arrowized_segments_display_columns = [
    "trips_hr_sch",
    "p50_mph",
    "p20_mph",
    "p80_mph",
    "route_short_name",
    "stop_pair_name",
    "segment_id",
    "route_id",
    "shape_id",
    arrowized_gdf.geometry.name,
]

In [ ]:
# Get study corridors to add onto the map
study_corridors = gpd.read_file("study_corridors_lines.geojson")
study_corridors["bus_lane"] = study_corridors["bus_lane"].fillna(0)
study_corridors.geometry = study_corridors.to_crs(
    geography_utils.CA_NAD83Albers_m
).buffer(100, cap_style="flat")
# Get colormap based on whether the analysis segment is a bus lane
DDS_GREY = "#d9d9d6"
DDS_BLUE = "#b1e4e3"
study_corridor_colormap = branca.colormap.StepColormap(
    colors=[DDS_GREY, DDS_BLUE], index=[0, 1]
)

### Create webmap

In [ ]:
SIGNAL_LEGEND_URL = "https://storage.googleapis.com/calitp-map-tiles/signal_legend.svg"
signal_colorscale = branca.colormap.step.Purples_05.scale(
    vmin=0, vmax=sjoined_signals_segments["trips_hr_sch"].max()
)
signal_folder = "signals_v12_34/"
# Study corridors
study_corridor_map = webmap_utils.set_state_export(
    study_corridors,
    subfolder=signal_folder,
    filename="study_corridors",
    cmap=study_corridor_colormap,
    color_col="bus_lane",
    map_title="Study Corridors",
)
# Speeds
speedmap = webmap_utils.set_state_export(
    arrowized_gdf[arrowized_segments_display_columns],
    subfolder=signal_folder,
    filename="speeds",
    cmap=rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE,
    color_col="p20_mph",
    cache_seconds=1,
    map_type="new_speedmap",
    legend_url=rt_utils.SPEEDMAP_LEGEND_URL,
    map_title="Speeds",
    existing_state=study_corridor_map["state_dict"],
)
# Signals
signal_speedmap = webmap_utils.set_state_export(
    buffered_signals[signals_with_transit_display_columns],
    subfolder=signal_folder,
    cmap=signal_colorscale,
    color_col="Trips/Hour",
    existing_state=speedmap["state_dict"],
    map_title=f"Signals with Approach Speeds {TARGET_DATE}",
    # legend_url=SIGNAL_LEGEND_URL,
    manual_centroid=[34.048108, -118.4183252],
)
signal_speedmap

### Get tabular data

In [ ]:
# Get signal-route grain data
signal_route_group = signals_segments_removed_duplicates.groupby(
    ["objectid", "route_short_name", "organization_name", "direction_id"]
)

signals_routes_frequency = signal_route_group["trips_hr_sch"].sum()
merged_signals_routes_frequency = signals_routes_frequency.reset_index().merge(
    caltrans_signals_filtered[
        [
            "tms_unit_type",
            "asset_sub_type",
            "location",
            "tms_id",
            "imms_id",
            "delegation_type",
            "leased_owned",
            "comment",
            "geometry",
        ]
    ],
    how="left",
    left_on="objectid",
    right_index=True,
    validate="many_to_one",
)
merged_geometry = gpd.GeoSeries(merged_signals_routes_frequency["geometry"]).to_crs(
    geography_utils.WGS84
)
merged_signals_routes_frequency["latitude"] = merged_geometry.y.round(5)
merged_signals_routes_frequency["longitude"] = merged_geometry.x.round(5)
merged_signals_routes_frequency.drop("geometry", axis=1).to_csv(
    "signals_routes.csv", index=False
)

In [ ]:
buffered_signals.head()["Trips/Hour"]

In [ ]:
# Save signal-grain data to a csv
caltrans_signals_filtered["latitude"] = caltrans_signals_filtered.geometry.y.round(5)
caltrans_signals_filtered["longitude"] = caltrans_signals_filtered.geometry.x.round(5)
caltrans_signals_filtered.rename(columns={"trips_hr_sch": "Trips/Hour"})[
    [*signals_with_transit_display_columns, "latitude", "longitude", "comment"]
].drop(caltrans_signals_filtered.geometry.name, axis=1).to_csv("signals_aggregated.csv")

In [ ]:
caltrans_signals_filtered["latitude"]